# RDD 고급 개념
- 이 장에서 다룰 핵심 주제
  - 집계와 키-값 형태의 RDD
  - 사용자 정의 파티셔닝
  - RDD 조인

In [0]:
myCollection = "Spark The Definitive Guide: Big Data Processing Made Simple".split(' ')
words = spark.sparkContext.parallelize(myCollection, 2)

## 키-값 형태의 기초(키-값 형태의 RDD)
- 메서드 이름에 ByKey가 있다면 PairRDD 타입(키-값 타입)만 사용 가능
  - RDD에 맵 연산을 수행해서 키-값 구조로 만들 수 있음

In [0]:
#(키, 값)
words.map(lambda word: (word.lower(), 1)).collect()

Out[15]: [('spark', 1),
 ('the', 1),
 ('definitive', 1),
 ('guide:', 1),
 ('big', 1),
 ('data', 1),
 ('processing', 1),
 ('made', 1),
 ('simple', 1)]

### keyBy
- 현재 값으로 부터 키를 생성하는 함수

In [0]:
#단어의 첫번째 문자를 키로 만들어 RDD 생성
keyword = words.keyBy(lambda word: word.lower()[0])
keyword.collect()

Out[19]: [('s', 'Spark'),
 ('t', 'The'),
 ('d', 'Definitive'),
 ('g', 'Guide:'),
 ('b', 'Big'),
 ('d', 'Data'),
 ('p', 'Processing'),
 ('m', 'Made'),
 ('s', 'Simple')]

### 값 매핑하기
- mapValues
- flatMapValues

In [0]:
#위에서 생성한 키에 값 매핑
keyword.mapValues(lambda word: word.upper()).collect()

Out[4]: [('s', 'SPARK'),
 ('t', 'THE'),
 ('d', 'DEFINITIVE'),
 ('g', 'GUIDE:'),
 ('b', 'BIG'),
 ('d', 'DATA'),
 ('p', 'PROCESSING'),
 ('m', 'MADE'),
 ('s', 'SIMPLE')]

In [0]:
#flatMap함수는 단어의 각 문자를 값으로 하도록 함
keyword.flatMapValues(lambda word: word.upper()).collect()

Out[5]: [('s', 'S'),
 ('s', 'P'),
 ('s', 'A'),
 ('s', 'R'),
 ('s', 'K'),
 ('t', 'T'),
 ('t', 'H'),
 ('t', 'E'),
 ('d', 'D'),
 ('d', 'E'),
 ('d', 'F'),
 ('d', 'I'),
 ('d', 'N'),
 ('d', 'I'),
 ('d', 'T'),
 ('d', 'I'),
 ('d', 'V'),
 ('d', 'E'),
 ('g', 'G'),
 ('g', 'U'),
 ('g', 'I'),
 ('g', 'D'),
 ('g', 'E'),
 ('g', ':'),
 ('b', 'B'),
 ('b', 'I'),
 ('b', 'G'),
 ('d', 'D'),
 ('d', 'A'),
 ('d', 'T'),
 ('d', 'A'),
 ('p', 'P'),
 ('p', 'R'),
 ('p', 'O'),
 ('p', 'C'),
 ('p', 'E'),
 ('p', 'S'),
 ('p', 'S'),
 ('p', 'I'),
 ('p', 'N'),
 ('p', 'G'),
 ('m', 'M'),
 ('m', 'A'),
 ('m', 'D'),
 ('m', 'E'),
 ('s', 'S'),
 ('s', 'I'),
 ('s', 'M'),
 ('s', 'P'),
 ('s', 'L'),
 ('s', 'E')]

### 키와 값 추출하기

In [0]:
keyword.keys().collect()

Out[6]: ['s', 't', 'd', 'g', 'b', 'd', 'p', 'm', 's']

In [0]:
keyword.values().collect()

Out[7]: ['Spark',
 'The',
 'Definitive',
 'Guide:',
 'Big',
 'Data',
 'Processing',
 'Made',
 'Simple']

### lookup
- 특정 키에 관한 결과 찾음

In [0]:
#키가 s인 값
keyword.lookup('s')

Out[20]: ['Spark', 'Simple']

### sampleByKey
- 근사치나 정확도를 이용해 키를 기반으로 RDD샘플 생성
- 특정 키를 부분 샘플링할 수 있음
- 해당 메서드는 RDD를 한 번만 처리하면서 간단한 무작위 샘플링을 사용함

In [0]:
import random

distinctChars = words.flatMap(lambda word: list(word.lower())).distinct().collect()

In [0]:
distinctChars

Out[21]: ['s',
 'p',
 'r',
 'h',
 'd',
 'i',
 'g',
 'b',
 'c',
 'l',
 'a',
 'k',
 't',
 'e',
 'f',
 'n',
 'v',
 'u',
 ':',
 'o',
 'm']

In [0]:
sampleMap = dict(map(lambda c: (c, random.random()), distinctChars))

In [0]:
sampleMap

Out[22]: {'s': 0.7936904503479406,
 'p': 0.5785935354075586,
 'r': 0.43445660462206803,
 'h': 0.7169604708460054,
 'd': 0.22146893050984584,
 'i': 0.2064943814602722,
 'g': 0.48235203607324517,
 'b': 0.8871165998134115,
 'c': 0.06025399003409204,
 'l': 0.6002817716207339,
 'a': 0.14384964835729097,
 'k': 0.24699504087909907,
 't': 0.5212045632858444,
 'e': 0.9141568362218496,
 'f': 0.42113602001446215,
 'n': 0.8659736989215989,
 'v': 0.48589216886256326,
 'u': 0.12244295105462766,
 ':': 0.3148236192239149,
 'o': 0.46635341871912694,
 'm': 0.8233393801557051}

In [0]:
words.map(lambda word: (word.lower()[0], word)).sampleByKey(True, sampleMap, 6).collect()

Out[11]: [('t', 'The'), ('m', 'Made'), ('s', 'Simple'), ('s', 'Simple')]

## 집계

In [0]:
chars= words.flatMap(lambda word: word.lower())
KVcharacters = chars.map(lambda letter: (letter, 1))

In [0]:
chars.take(5)

Out[28]: ['s', 'p', 'a', 'r', 'k']

In [0]:
KVcharacters.take(5)

Out[26]: [('s', 1), ('p', 1), ('a', 1), ('r', 1), ('k', 1)]

In [0]:
def maxFunc(left, right):
  return max(left, right)

def addFunc(left, right):
  return left+right

In [0]:
nums = sc.parallelize(range(1,31),5)

### countByKey
- 각 키의 아이템 수를 구하고 로컬 맵으로 결과를 수집

In [0]:
KVcharacters.countByKey()

Out[32]: defaultdict(int,
 {'s': 4,
 'p': 3,
 'a': 4,
 'r': 2,
 'k': 1,
 't': 3,
 'h': 1,
 'e': 7,
 'd': 4,
 'f': 1,
 'i': 7,
 'n': 2,
 'v': 1,
 'g': 3,
 'u': 1,
 ':': 1,
 'b': 1,
 'o': 1,
 'c': 1,
 'm': 2,
 'l': 1})

### 집계 연산 구현 방식 이해하기
- 키-값 형태의 PairRDD를 생성하는 몇 가지 방법이 있음
  - groupBy
  - reduce
- 이때 구현 방식은 Job의 안정성을 위해 매우 중요함

#### groupByKey
- 각 키에 대한 <strong>값의 크기가 일정</strong>하고 익스큐터에 <strong>할당된 메모리에서 처리 가능할 정도</strong>의 크기라면 해당 메서드 사용
  - 모든 익스큐터에서 함수를 적용하기 전에 <strong>해당 키와 관련된 모든 값을 메모리로 읽어 들임</strong>
  - 따라서 만약 심각하게 치우쳐진 키가 있다면 일부 파티션이 엄청난 양의 값을 가질 수 있음(out of memory)

In [0]:
from functools import reduce
#각 키에 1의 값을 매핑했었음
KVcharacters.groupByKey().map(lambda row: (row[0],reduce(addFunc, row[1]))).collect()

Out[37]: [('s', 4),
 ('p', 3),
 ('r', 2),
 ('h', 1),
 ('d', 4),
 ('i', 7),
 ('g', 3),
 ('b', 1),
 ('c', 1),
 ('l', 1),
 ('a', 4),
 ('k', 1),
 ('t', 3),
 ('e', 7),
 ('f', 1),
 ('n', 2),
 ('v', 1),
 ('u', 1),
 (':', 1),
 ('o', 1),
 ('m', 2)]

#### reduceByKey
- 작업 부하를 줄이려는 경우 적합
- <strong>각 파티션에서 리듀스 작업</strong>을 수행하기 때문에 안정적이고 <strong>모든 값을 메모리에 유지하지 않아도 됨</strong>
- 또한 최종 리듀스 과정을 제외한 <strong>모든 작업은 개별 워커에서 처리</strong>하므로 연산 중 셔플 발생 X

In [0]:
KVcharacters.reduceByKey(addFunc).collect()

Out[38]: [('s', 4),
 ('p', 3),
 ('r', 2),
 ('h', 1),
 ('d', 4),
 ('i', 7),
 ('g', 3),
 ('b', 1),
 ('c', 1),
 ('l', 1),
 ('a', 4),
 ('k', 1),
 ('t', 3),
 ('e', 7),
 ('f', 1),
 ('n', 2),
 ('v', 1),
 ('u', 1),
 (':', 1),
 ('o', 1),
 ('m', 2)]

### 기타 집계 메서드

#### aggregate
- 파티션 기준 집계

In [0]:
nums.aggregate(0, maxFunc, addFunc)#시작값, 파티션 내에 수행될 함수, 모든 파티션에 걸쳐 수행될 함수

Out[39]: 90

#### treeAggregate
- 위 aggregate는 드라이버에서 최종 집계를 수행하므로 익스큐터 결과가 너무 크면 out of memory
- 그래서 <strong>익스큐터끼리 트리를 형성해서 집계 처리의 일부 하위 과정을 push down</strong> 방식으로 수행하는 것이 해당 함수
- 이렇게 집계 처리를 <strong>여러 단계</strong>로 구성해서 드라이버의 메모리를 모두 소비하는 현상을 막음

In [0]:
nums.treeAggregate(0, maxFunc, addFunc, depth=3)

Out[42]: 90

#### aggregateByKey
- 파티션 대신 키를 기준으로 집계

In [0]:
KVcharacters.aggregateByKey(0, addFunc, maxFunc).collect()

Out[43]: [('s', 3),
 ('p', 2),
 ('r', 1),
 ('h', 1),
 ('d', 2),
 ('i', 4),
 ('g', 2),
 ('b', 1),
 ('c', 1),
 ('l', 1),
 ('a', 3),
 ('k', 1),
 ('t', 2),
 ('e', 4),
 ('f', 1),
 ('n', 1),
 ('v', 1),
 ('u', 1),
 (':', 1),
 ('o', 1),
 ('m', 2)]

#### combineByKey
- 키를 기준으로 연산을 수행하며 파라미터로 사용된 함수에 따라 값을 병합함
- 그런다음 여러 결괏값을 병합해 결과 반환
- 사용자 정의 파티셔너를 사용해 출력 파티션 수를 지정할 수도 있음

In [0]:
def valToCombiner(v):
  return [v]

def mergeValuesFunc(v, valToAppend):
  v.append(valToAppend)
  return v

def mergeCombinerFunc(v1, v2):
  return v1+v2

outputPartitions=6

In [0]:
KVcharacters.combineByKey(valToCombiner, mergeValuesFunc, mergeCombinerFunc,outputPartitions).collect()

Out[47]: [('s', [1, 1, 1, 1]),
 ('d', [1, 1, 1, 1]),
 ('l', [1]),
 ('v', [1]),
 (':', [1]),
 ('p', [1, 1, 1]),
 ('r', [1, 1]),
 ('c', [1]),
 ('k', [1]),
 ('t', [1, 1, 1]),
 ('n', [1, 1]),
 ('u', [1]),
 ('o', [1]),
 ('h', [1]),
 ('i', [1, 1, 1, 1, 1, 1, 1]),
 ('g', [1, 1, 1]),
 ('b', [1]),
 ('a', [1, 1, 1, 1]),
 ('e', [1, 1, 1, 1, 1, 1, 1]),
 ('f', [1]),
 ('m', [1, 1])]

#### foldByKey
- 결합 함수와 항등원(어떤 원소와 연산을 취해도 자기 자신이 되게 하는 원소)인 제로값을 이용해 각 키의 값을 병합

In [0]:
KVcharacters.foldByKey(0,addFunc).collect()

Out[48]: [('s', 4),
 ('p', 3),
 ('r', 2),
 ('h', 1),
 ('d', 4),
 ('i', 7),
 ('g', 3),
 ('b', 1),
 ('c', 1),
 ('l', 1),
 ('a', 4),
 ('k', 1),
 ('t', 3),
 ('e', 7),
 ('f', 1),
 ('n', 2),
 ('v', 1),
 ('u', 1),
 (':', 1),
 ('o', 1),
 ('m', 2)]